In [1]:
import pandas as pd
import numpy as np
from random import sample
from datetime import datetime

In [2]:
path_name = "/home/user/Projects/owascrp/Outputs/Merged/"
file_name = "owa_merged_116_157.csv"

data_raw = pd.read_csv(path_name+file_name, delimiter=";")

/home/user/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_raw.head(2)

,Unnamed: 0,Auction,Auction_Close,Auction_No,By,Color,Condition,Date,Date_High,Date_Low,...,Publication,Size,Size_Cm,Size_Cm_H,Size_Cm_W,Size_Inches,Sold_For,Subject,Text,Title
0,0,116 - 9/20/2006,9/20/2006,116,Heinrich Bunting,hand color,B,1582,NaN,1582,...,,7 x 11.7 inches 17.8 x 29.8 cm,17.8x29.8,29.8,17.8,7x11.7,600,Title Pages,\n\nTitle page from Bunting's \nItinerarium Sa...,"Lot #1: ""Itinerarium Sacrae Scripturae Das ist..."
1,1,116 - 9/20/2006,9/20/2006,116,,hand color,,1595-1642,1642,1595,...,,0 x 0 inches 0.0 x 0.0 cm,0.0x0.0,0.0,0.0,0x0,140,Title Pages,\n\n1) This title page was for Johannes Blaeu...,"Lot #2: ""[Lot of 2]"" \n"


In [7]:
#data_raw[data_raw["Auction_No"]==153.00].head(2)

## Data Cleaning

### Eliminating Missing

In [8]:
 data_raw.rename(columns = {'Unnamed: 0':'Index'}, inplace = True)

In [9]:
print('---- In total %i rows ------' % len(data_raw))
for col in data_raw.columns:
    if sum(pd.isnull(data_raw[col]))>0:
      print("%i Null-Werte in Spalte %s" %(sum(pd.isnull(data_raw[col])), col))

---- In total 34571 rows ------
4 Null-Werte in Spalte Index
32501 Null-Werte in Spalte Date_High
3 Null-Werte in Spalte Date_Low
4 Null-Werte in Spalte Estimate
4 Null-Werte in Spalte Estimate_High
4 Null-Werte in Spalte Estimate_Low
4 Null-Werte in Spalte Image
4 Null-Werte in Spalte Lot
4 Null-Werte in Spalte Publication
8 Null-Werte in Spalte Size
8 Null-Werte in Spalte Size_Cm
8 Null-Werte in Spalte Size_Cm_H
8 Null-Werte in Spalte Size_Cm_W
8 Null-Werte in Spalte Size_Inches
8 Null-Werte in Spalte Sold_For
8 Null-Werte in Spalte Subject
8 Null-Werte in Spalte Text
8 Null-Werte in Spalte Title


In [10]:
#data_raw[data_raw["Size"].isnull()].head(2)

In [11]:
data_cleaner = data_raw[data_raw["Subject"].notnull()]
print('---- In total %i rows ------' % len(data_cleaner))

---- In total 34563 rows ------


In [12]:
data_cleaner = data_cleaner[data_cleaner["Date_Low"].notnull()]
print('---- In total %i rows ------' % len(data_cleaner))

---- In total 34560 rows ------


In [13]:
print('---- In total %i rows ------' % len(data_cleaner))
for col in data_cleaner.columns:
    if sum(pd.isnull(data_cleaner[col]))>0:
      print("%i Null-Werte in Spalte %s" %(sum(pd.isnull(data_cleaner[col])), col))

---- In total 34560 rows ------
32494 Null-Werte in Spalte Date_High


## Replacing wrong entries

In [14]:
#data_cleaner.columns

---- Correction of Auction Close Date ---- 

In [15]:
#np.unique(data_cleaner["Index"])
#np.unique(data_cleaner["Auction"])
#np.unique(data_cleaner["Auction_No"])
#np.unique(data_cleaner["Auction_Close"])

In [16]:
data_cleaner2 = data_cleaner
def make_date (x):
    return datetime.strptime(x, '%m/%d/%Y')
data_cleaner2["Auction_Close"]=data_cleaner2["Auction_Close"].apply(make_date)

In [17]:
#np.unique(data_cleaner["Auction_Close"])

----- Correction of Date_Low -----

In [18]:
#data_cleaner[data_cleaner["Date_Low"]=='`926']
#len(data_cleaner[data_cleaner["Date_Low"]!='`926'])
data_cleaner = data_cleaner[data_cleaner["Date_Low"]!='`926']

In [19]:
#data_cleaner[data_cleaner["Date_Low"] == 9999]

In [20]:
def left_4 (x):
    x = str(x)
    if len(x[:4])>3: 
        return int(x[:4])
    else:
        return int(9999)
data_cleaner["Date_Low"]=data_cleaner["Date_Low"].apply(left_4)
#np.unique(data_cleaner["Date_Low"])

/home/user/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
#len(data_cleaner[data_cleaner["Date_Low"]==9999])
data_cleaner = data_cleaner[data_cleaner["Date_Low"]!=9999]


---- Correction of Condition ---- 

In [22]:
np.unique(data_cleaner["Condition"])

array([' ', '     ', ' +A  ', ' +B  ', ' +C  ', ' A   ', ' A + ', ' A+  ',
       ' B   ', ' B+  ', ' C   ', ' C+  ', ' D   '], dtype=object)

In [23]:
#data_cleaner[data_cleaner["Condition"]== ' A + ']

In [24]:
#data_cleaner[data_cleaner["Condition"]== ' +A  '].head(2)

In [25]:
data_cleaner[data_cleaner["Index"] == 47][data_cleaner[data_cleaner["Index"] == 47]["Auction_No"]== 125]

,Index,Auction,Auction_Close,Auction_No,By,Color,Condition,Date,Date_High,Date_Low,...,Publication,Size,Size_Cm,Size_Cm_H,Size_Cm_W,Size_Inches,Sold_For,Subject,Text,Title
8176,47,125 - 9/24/2008,2008-09-24,125,Scherer,hand color,A +,1703,NaN,1703,...,Atlas Novus,14 x 9 inches 35.6 x 22.9 cm,35.6x22.9,22.9,35.6,14x9,1500,World - Polar,\n\nThis very graphic map of the world is pres...,"Lot #48: ""Repraesentatio Geographica Itineris ..."


In [26]:
def cond_repair_1(x):
    return x.replace(" ", "")
data_cleaner["Condition"]=data_cleaner["Condition"].apply(cond_repair_1)
np.unique(data_cleaner["Condition"])

array(['', '+A', '+B', '+C', 'A', 'A+', 'B', 'B+', 'C', 'C+', 'D'], dtype=object)

In [27]:
def cond_repair_2(x):
    if len(x)<1:
        x='None'
    else:
        if x[0]=='+':
            x=x[1]+x[0]
    return x
data_cleaner["Condition"]=data_cleaner["Condition"].apply(cond_repair_2)

In [28]:
#np.unique(data_cleaner["Condition"])

In [29]:
data_cleaner.columns

Index(['Index', 'Auction', 'Auction_Close', 'Auction_No', 'By', 'Color',
       'Condition', 'Date', 'Date_High', 'Date_Low', 'Estimate',
       'Estimate_High', 'Estimate_Low', 'Image', 'Lot', 'Publication', 'Size',
       'Size_Cm', 'Size_Cm_H', 'Size_Cm_W', 'Size_Inches', 'Sold_For',
       'Subject', 'Text', 'Title'],
      dtype='object')

--- Correction of Estimates and Sold_For -----

In [30]:
def to_int (x):
    return int(x)
data_cleaner["Estimate_High"] =data_cleaner["Estimate_High"].apply(to_int)

In [31]:
data_cleaner["Estimate_Low"] =data_cleaner["Estimate_Low"].apply(to_int)

In [32]:
data_cleaner["Sold_For"] =data_cleaner["Sold_For"].apply(to_int)


----- Correction of Cartographers (By) -----

In [33]:
def clean_carto(x):
    x = x.strip()
    if x == '' or x == '[blank]':
        x = 'Unknown'
    return x
data_cleaner["By"] = data_cleaner["By"].apply(clean_carto)
#data_cleaner[ data_cleaner["By"] == 'Unknown']

In [34]:
len(np.unique(data_cleaner["By"]))

1883

In [35]:
np.unique(data_cleaner["By"])

array(['A. A. Grant', 'A. Anderson', 'A. Boschke', ...,
       'jean Baptiste Poirson', 'le jeune Basset',
       'Étienne Geoffroy St. Hilaire/Georges Cuvier'], dtype=object)

### Graphs and Analysis

In [39]:
data_cleaner.describe()

,Index,Auction_No,Date_Low,Estimate_High,Estimate_Low,Lot,Size_Cm_H,Size_Cm_W,Sold_For
count,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000
mean,428.050149,135.391527,1744.683827,715.282577,572.502098,429.050149,31.910174,38.321388,-2044.960963
std,252.379831,11.993218,106.623170,1684.671711,1361.288388,252.379831,18.819407,23.494565,4573.601357
min,0.000000,116.000000,1250.000000,15.000000,10.000000,1.000000,0.000000,0.000000,-10000.000000
25%,210.000000,125.000000,1671.000000,200.000000,150.000000,211.000000,19.600000,22.900000,47.000000
50%,424.000000,135.000000,1750.000000,325.000000,250.000000,425.000000,30.500000,36.100000,150.000000
75%,640.000000,146.000000,1835.000000,600.000000,475.000000,641.000000,42.000000,50.800000,300.000000
max,994.000000,157.000000,2010.000000,90000.000000,70000.000000,995.000000,335.300000,469.900000,52500.000000


In [37]:
a = data_cleaner[data_cleaner["Sold_For"]>0]["Sold_For"]
b= data_cleaner[data_cleaner["Sold_For"]>0]["Date_Low"]
np.corrcoef(a, b)[0, 1]

-0.12361775374100703

Fraction of Sales per Century

In [38]:
data_to_graph = data_cleaner[["Auction_No", "Date_Low", "Sold_For"]]

def data_century(x):
    x = str(x+100)
    x = x[0]+x[1]+'th'
    return x

def data_sold(y):
    z= 0
    if y > 0:
        z= 1
    return z

def data_zero (a):
    if a > 0:
        return a
    else:
        return 0

data_to_graph["Century"] = data_to_graph["Date_Low"].apply(data_century)
data_to_graph["Cnt"] = 1
data_to_graph["Sold"] = data_to_graph["Sold_For"].apply(data_sold)
data_to_graph["Sold_For"] = data_to_graph["Sold_For"].apply(data_zero)
data_to_graph = data_to_graph[data_to_graph["Sold_For"] > 0].sort(['Auction_No', 'Date_Low'], ascending=[1, 0])
data_to_graph

/home/user/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/user/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/user/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

,Auction_No,Date_Low,Sold_For,Century,Cnt,Sold
889,116,1998,110,20th,1,1
900,116,1998,80,20th,1,1
890,116,1996,90,20th,1,1
899,116,1995,35,20th,1,1
898,116,1994,30,20th,1,1
896,116,1992,42,20th,1,1
895,116,1990,50,20th,1,1
894,116,1989,210,20th,1,1
893,116,1987,75,20th,1,1
547,116,1977,100,20th,1,1


In [110]:
data_to_graph_group = data_to_graph.groupby(['Auction_No'], as_index=False)
data_to_graph_group_2 = data_to_graph_group.agg({"Sold" : np.mean, "Sold_For" : np.mean, "Cnt" : np.sum})
data_to_graph_group_2.head(2)

,Auction_No,Sold_For,Cnt,Sold
0,116,366.217536,901,0.855716
1,117,347.726872,908,0.822687


In [2]:
"""
data_to_graph_2 = data_cleaner[["Sold_For", "By"]]
data_to_graph_2 = data_to_graph_2[data_to_graph_2["Sold_For"] > 0]
data_to_graph_2 = data_to_graph_2[data_to_graph_2["By"] !='']
data_to_graph_group_3 = data_to_graph_2.groupby(['By'], as_index=False)
data_to_graph_group_3.size()
"""

'\ndata_to_graph_2 = data_cleaner[["Sold_For", "By"]]\ndata_to_graph_2 = data_to_graph_2[data_to_graph_2["Sold_For"] > 0]\ndata_to_graph_2 = data_to_graph_2[data_to_graph_2["By"] !=\'\']\ndata_to_graph_group_3 = data_to_graph_2.groupby([\'By\'], as_index=False)\ndata_to_graph_group_3.size()\n'

In [112]:
"""from bokeh.charts import Bar, output_file, show, hplot

#Bar Plots of sell rate and avg price
s1 = Bar(data_to_graph_group_2, label='Auction_No', values='Sold',  title="Sell Rate", ylabel="Sell Rate")
s2 = Bar(data_to_graph_group_2, label='Auction_No', values='Sold_For', title="Avg Price",color="navy", 
        ylabel="Avg Price")

output_file("test_owa_stacked_bar.html")

# put all the plots in an HBox
p = hplot(s1, s2)

# show the results
show(p)

"""




/home/user/anaconda3/lib/python3.5/site-packages/bokeh/charts/_attributes.py:78: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  df = df.sort(columns=columns)
